<a href="https://colab.research.google.com/github/weichen-liao/Qard_lwc/blob/main/Qard_Case_Study_test_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description
This notebook explores the performance of different NER methods

These methods includes Spacy and camembert-ner

The difference on languages are also considered(EN and FR)

The conclusion is: **camembert-ner > Spacy-EN > Spacy-FR**

### Install Dependencies

In [ ]:
! apt-get install poppler-utils
! apt-get install tesseract-ocr

! pip install pdf2image
! pip install pytesseract
! pip install pip setuptools wheel
! pip install spacy
! pip install googletrans
! pip install transformers
! pip install sentencepiece

! python -m spacy download en_core_web_sm
! python -m spacy download fr_core_news_sm

### Upload the pdf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test1.pdf to test1.pdf


In [3]:
! mkdir pdf
! mv test1.pdf pdf/test1.pdf
! mkdir img

### PDF2image
images are saved as ppm format
A PPM file is a 24-bit color image formatted using a text format. It stores each pixel with a number from 0 to 65536, which specifies the color of the pixel. PPM files also store the image height and width, whitespace data, and the maximum color value.


In [5]:
from pdf2image import convert_from_path, convert_from_bytes

images_from_path = convert_from_path('pdf/test1.pdf', output_folder='img/')

### OCR pytesseract

In [6]:
import pytesseract
from PIL import Image

In [7]:
! ls img/

886d9d4a-daea-4571-9e3f-e035361ab0b0-01.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-02.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-03.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-04.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-05.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-06.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-07.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-08.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-09.ppm
886d9d4a-daea-4571-9e3f-e035361ab0b0-10.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-01.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-02.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-03.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-04.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-05.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-06.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-07.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-08.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-09.ppm
8b453f0b-c20e-4329-bd4e-0c43b2525f13-10.ppm


In [8]:
text = pytesseract.image_to_string(Image.open('img/886d9d4a-daea-4571-9e3f-e035361ab0b0-02.ppm'))
print(text)

Les soussignés :

Monsieur TUREMEN Suleyman

Né le : 17 Novembre 1968 a CIVRIL (Turquie)

Domicilié : 7, rue des Pommiers — 76410 SAINT AUBIN LES ELBEUF
Nationalité : Turque

Titre de séjour N° 750252713 — valable jusqu’au 02 Mai 2014

Monsieur YILANCI Sezai

Né le : 4 Décembre 1974 a CIVRIL (Turquie)

Domicili¢ ; 5, place des Etoiles Escal.3 — 93230 ROMAINVILLE
Nationalité : Turque

Titre de séjour N° 763034864 — valable jusqu’an 22 Janvier 2011

Monsieur ERDOGDU Almet

Né le : 17 Novembre 1972 a CIVRIL (Turquie)

Domicilié : Cité Youri Gargarine — Batiment J — 93230 - ROMAINVILLE
Nationalité : Turque

Ont convenu ce qui suit :

Article 1—- FORME

Tl est formé entre les soussignés une société 4 responsabilité limitée, qui sera régie par les lois en vigueur ainsi
qne par les présents statuts.

Article 2 - OBJET SOCIAL

La société a pour objet toutes activités d’entreprise de magonnerie neuf et rénovation et généralement toutes
opérations commerciales, financitres, mobilitres ou immobil

### Spacy NER

the in-built MER French model of Spacy is very inaccurate and limited, for example it can tell Apple from **Apple est une entreprise internationale** but can't tell Tesla from **Tesla est une entreprise international**. when seeing an unseen company name, it would most likely fail.

In [13]:
import spacy
from spacy import displacy
import fr_core_news_sm
# import en_core_web_sm

def show_ents(doc):
  if doc.ents:
    for ent in doc.ents:
      print('|'.join([ent.text.strip(), ent.label_, str(spacy.explain(ent.label_))]))

text1 = 'Apple est une entreprise internationale'
text2 = 'Tesla est une entreprise internationale'
text3 =  'TUR - MAN est une entreprise internationale'
text4 = 'La dénomination de la société est : TUR - MAN'
nlp = fr_core_news_sm.load()
for t in [text1, text2, text3, text4]:
  doc = nlp(t)
  show_ents(doc)
  # displacy.render(doc, style='ent',jupyter=True, options={'ents': ['ORG', 'PRODUCT']})
  # displacy.render(doc, style='ent',jupyter=True)


Apple|ORG|Companies, agencies, institutions, etc.
Tesla|LOC|Non-GPE locations, mountain ranges, bodies of water
TUR|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
MAN|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
TUR|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art


Let's try the English NER model

In [11]:
import en_core_web_sm
text1 = 'Apple is an international company'
text2 = 'Tesla is an international company'
text3 = 'TUR - MAN is an international company'
text4 = 'NAME The name of the company is: TUR - MAN'
nlp = en_core_web_sm.load()
for t in [text1, text2, text3, text4]:
  doc = nlp(t)
  show_ents(doc)
  # displacy.render(doc, style='ent',jupyter=True)

Apple|ORG|Companies, agencies, institutions, etc.
Tesla|ORG|Companies, agencies, institutions, etc.
TUR - MAN|ORG|Companies, agencies, institutions, etc.
TUR - MAN|PRODUCT|Objects, vehicles, foods, etc. (not services)


I translated the page into English using GoogleTranslate, the NER is still a big mess, to many False Positives

In [14]:
text_translated = """
Each share gives its owner an equal right in your shareholder's profits and in all assets.
social. Any share gives the right to a vote in all votes and deliberations.

 

Any transfer of shares must be recorded by a notarial deed or under private signature to be enforceable 8 days
company, it must be served by bailiff's writ or be accepted by it in a notarial deed.

The shares are freely transferable between partners. They cannot be sold to foreign third parties.
company, only with the consent of the majority of associates representing at least three / quarters of the shares
social, this majority being determined taking into account the person and the shares of the cbdant lassoé. the
consent is requested in accordance with the procedure provided for by law. The shares are freely transferable
by vale of succession or in the event of community liquidation between spouses, and freely transferable between
spouses and between ascendants.

acquisition by the spouse, subsequent to the realization of the ownership or the purchase, as a partner
under the conditions set by fartlele 1832-2 of the Civil Code, is subject to the consent of the Maforité des
associated, representing at least bes three / quarters of the soda shares.

The company has given its consent to a plan to pledge shares, either by notification of
its decision & the interested party, either by default of response within a period of three months & from the request, this
consent will result in the consent of the assignee in the event of formalization of shares, according to the
provisions of Article 207, paragraphs 1 ° of the Civil Code, 3 unless the company prefers, after transfer, to redeem
without dei the living shares reduce the capital.

Article 31 ~ RESPONSIBILITY OF PARTNERS
Assotids are only liable to third parties up to the amount of their contribution.

they are, however, voluntarily respansabies for five years, vis-a-vis the thirds, of the value attributed to
spposts in natare.

This responsibility only comes into play if there has been no intervention by a commissioner in an apartment or
when [the value retained by your partner is different from that proposed by the contribution auditor.

In addition, you are reminded that, in accordance with your law, forsque reorganization or judicial liquidation
an Insufficiency of assets, the court may, in the event of a mismanagement that contributed to this
insufficient, decide that the debts of the company will be borne in whole or in part, by the directors of
law or de facto, or some of them, with or without sofidarity.

Article 32 ~ MANAGEMENT
Modatted:

The sockété is managed by one or more natural persons, associates or not, appointed with or without.
time limit. In the latter case, the geranite (s) are always eligible.

The managers are appointed by deviation from partners representing more than half of the shares.

In compensation for his duties, each manager is entitled to a fixed, proportional or mixed salary, the amount of which
amount and modalities of palerment health determined by collective ardinary decision of the partners.
"""
doc = nlp(text_translated)
show_ents(doc)
# displacy.render(doc, style='ent',jupyter=True)

Any share gives the right to|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
it must be served|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
be accepted by it in|ORG|Companies, agencies, institutions, etc.
between partners|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
They cannot be sold to foreign third parties|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
and freely|PER|Named person or family.
between|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
as|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
under the conditions set by fartlele|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
-2 of the Civil Code|MISC|Miscellaneous entities, e.g. events, nationalities, products or works of art
is subject to the consent of the Maforité|MISC|Miscellaneous e

***Another problem of translating the document into English before using Spacy NER, is that it can't be scaled as translation APIs are costly. ***

### **camembert-ner NER**
[camembert-ner] is a NER model that was fine-tuned from camemBERT on wikiner-fr dataset. Model was trained on wikiner-fr dataset (~170 634 sentences). Model was validated on emails/chat data and overperformed other models on this type of data specifically. In particular the model seems to work better on entity that don't start with an upper case.

In [15]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

nlp_tf = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")


Downloading:   0%|          | 0.00/269 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

the performance seems better

In [17]:
text1 = 'Apple est une entreprise internationale'
text2 = 'Tesla est une entreprise internationale'
text3 =  'TUR - MAN est une entreprise internationale'
text4 = 'La dénomination de la société est : TUR - MAN'

for item in nlp_tf(text.replace(' - ', '-')):
  # if item['entity_group']=='ORG':
    print(item)

{'entity_group': 'PER', 'score': 0.99780595, 'word': 'TUREMEN Suleyman', 'start': 26, 'end': 43}
{'entity_group': 'LOC', 'score': 0.9987271, 'word': 'CIVRIL', 'start': 71, 'end': 78}
{'entity_group': 'LOC', 'score': 0.998704, 'word': 'Turquie', 'start': 80, 'end': 87}
{'entity_group': 'LOC', 'score': 0.89125204, 'word': 'rue des Pommiers', 'start': 104, 'end': 121}
{'entity_group': 'LOC', 'score': 0.99848354, 'word': 'SAINT AUBIN LES ELBEUF', 'start': 129, 'end': 152}
{'entity_group': 'LOC', 'score': 0.7559233, 'word': 'Turque', 'start': 166, 'end': 173}
{'entity_group': 'PER', 'score': 0.9979638, 'word': 'YILANCI Sezai', 'start': 244, 'end': 258}
{'entity_group': 'LOC', 'score': 0.99869937, 'word': 'CIVRIL', 'start': 285, 'end': 292}
{'entity_group': 'LOC', 'score': 0.9986891, 'word': 'Turquie', 'start': 294, 'end': 301}
{'entity_group': 'LOC', 'score': 0.9725759, 'word': 'place des Etoiles Escal.3', 'start': 318, 'end': 344}
{'entity_group': 'LOC', 'score': 0.8613232, 'word': '3230 R